# How to add examples to the prompt for query analysis

# 1. Install Dependencies

In [1]:
%pip install -qU langchain-core langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.3/414.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.8 MB/s eta 0:00:00


In [2]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

# Optional, uncomment to trace runs with LangSmith. Sign up here: https://smith.langchain.com.
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

··········


# 2. Define the Query schema

We'll define a query schema that we want our model to output. To make our query analysis a bit more interesting, we'll add a sub_queries field that contains more narrow questions derived from the top level question.

1. Import necessary dependencies

2. sub_queries_description String

This string provides instructions for generating sub-queries.

It explains that if the original question contains multiple distinct sub-questions or if there are generic questions that would be helpful to answer, a list of sub-questions should be created.

It emphasizes that the list should be comprehensive, cover all parts of the original question, and that redundancy is acceptable.

It also states that the sub-questions should be as narrowly focused as possible.

3. Search Pydantic Model

Search(BaseModel): Defines a Pydantic model called Search.

query: str = Field(...): Specifies that the model has a field named query, which is a string.

...: Indicates that this field is required.

description: Provides a description of the field, which is used to explain the purpose of the field.

sub_queries: List[str] = Field(...): Specifies that the model has a field named sub_queries, which is a list of strings.

default_factory=list: Sets the default value of the field to an empty list.

description: Uses the sub_queries_description string as the description for this field.

publish_year: Optional[int] = Field(...): Specifies that the model has an optional field named publish_year, which is an integer.

None: Sets the default value of the field to None.

description: Provides a description of the field.

Summary

This code defines a Pydantic model that represents a search query for a database of tutorial videos. The model includes fields for the primary query, sub-queries, and the publication year. The sub_queries_description string provides instructions for generating sub-queries, which can be used to improve the search results.

In [3]:
from typing import List, Optional

from pydantic import BaseModel, Field

sub_queries_description = """\
If the original question contains multiple distinct sub-questions, \
or if there are more generic questions that would be helpful to answer in \
order to answer the original question, write a list of all relevant sub-questions. \
Make sure this list is comprehensive and covers all parts of the original question. \
It's ok if there's redundancy in the sub-questions. \
Make sure the sub-questions are as narrowly focused as possible."""


class Search(BaseModel):
    """Search over a database of tutorial videos about a software library."""

    query: str = Field(
        ...,
        description="Primary similarity search query applied to video transcripts.",
    )
    sub_queries: List[str] = Field(
        default_factory=list, description=sub_queries_description
    )
    publish_year: Optional[int] = Field(None, description="Year video was published")

# 3. Query generation

1. Import necessary dependencies

2. System Message

This defines the system message for the LLM.

It sets the LLM's role as an expert in converting user questions into database queries.

It specifies that the LLM has access to a database of tutorial videos.

It instructs the LLM to return a list of database queries optimized for retrieving relevant results.

Key Instruction: It tells the LLM not to rephrase acronyms or unfamiliar words.

3. Prompt Template

This creates a ChatPromptTemplate with:

("system", system): The system message created earlier.

MessagesPlaceholder("examples", optional=True): A placeholder for examples, which can be dynamically inserted into the prompt. The optional=True argument means that the examples are not required.

("human", "{question}"): A human message with a placeholder for the user's question.

4. LLM Initialization

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0): Initializes the OpenAI chat model with the "gpt-4o-mini" model and a temperature of 0 (making the output deterministic).

structured_llm = llm.with_structured_output(Search): Creates a version of the LLM that is configured to return structured output in the form of the Search Pydantic model (defined in the previous explanation).

5. query_analyzer Chain Creation

{"question": RunnablePassthrough()}: This creates a dictionary that passes the "question" key from the input through to the next runnable in the chain.

|: This is the LangChain "pipe" operator, which chains together runnables.

prompt: The prompt template created earlier.

structured_llm: The LLM configured to return structured output.

Summary:

This code sets up a LangChain chain that takes a user's question and uses an LLM to generate a structured search query. The LLM is configured to return the output in the form of a Search Pydantic model, which includes the primary query, sub-queries, and publication year. This allows for more structured and controlled interaction with the LLM, making it easier to parse and use the output in downstream applications.

In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

system = """You are an expert at converting user questions into database queries. \
You have access to a database of tutorial videos about a software library for building LLM-powered applications. \
Given a question, return a list of database queries optimized to retrieve the most relevant results.

If there are acronyms or words you are not familiar with, do not try to rephrase them."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        MessagesPlaceholder("examples", optional=True),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm = llm.with_structured_output(Search)
query_analyzer = {"question": RunnablePassthrough()} | prompt | structured_llm

In [5]:
query_analyzer.invoke(
    "what's the difference between web voyager and reflection agents? do both use langgraph?"
)

Search(query='difference between web voyager and reflection agents', sub_queries=['What are web voyager agents?', 'What are reflection agents?', 'How do web voyager agents work?', 'How do reflection agents work?', 'Do both web voyager and reflection agents use langgraph?', 'What is langgraph?', 'What are the use cases for web voyager agents?', 'What are the use cases for reflection agents?'], publish_year=None)

# 4. Adding examples and tuning the prompt

Purpose:

These examples are designed to be used as few-shot examples for a language model (LLM). They demonstrate how to convert natural language questions into structured search queries using the Search Pydantic model. By providing these examples to the LLM, you can help it learn to generate structured search queries for new questions.



In [6]:
examples = []

In [7]:
question = "What's chat langchain, is it a langchain template?"
query = Search(
    query="What is chat langchain and is it a langchain template?",
    sub_queries=["What is chat langchain", "What is a langchain template"],
)
examples.append({"input": question, "tool_calls": [query]})

In [8]:
question = "How to build multi-agent system and stream intermediate steps from it"
query = Search(
    query="How to build multi-agent system and stream intermediate steps from it",
    sub_queries=[
        "How to build multi-agent system",
        "How to stream intermediate steps from multi-agent system",
        "How to stream intermediate steps",
    ],
)

examples.append({"input": question, "tool_calls": [query]})

In [9]:
question = "LangChain agents vs LangGraph?"
query = Search(
    query="What's the difference between LangChain agents and LangGraph? How do you deploy them?",
    sub_queries=[
        "What are LangChain agents",
        "What is LangGraph",
        "How do you deploy LangChain agents",
        "How do you deploy LangGraph",
    ],
)
examples.append({"input": question, "tool_calls": [query]})

Now we need to update our prompt template and chain so that the examples are included in each prompt. Since we're working with OpenAI function-calling, we'll need to do a bit of extra structuring to send example inputs and outputs to the model. We'll create a tool_example_to_messages helper function to handle this for us

1. Import necessary dependencies

2. tool_example_to_messages Function:

Purpose: Takes a dictionary (example) representing a conversation example and converts it into a list of BaseMessage objects.

Input:

example: A dictionary containing the conversation input (example["input"]), tool calls (example["tool_calls"]), and optionally tool outputs (example["tool_outputs"]).

Steps:

Human Message: Creates a HumanMessage object from the example["input"] and adds it to the messages list. This represents the user's input.

Tool Call Conversion:

Iterates through example["tool_calls"].

For each tool_call, it creates a dictionary (openai_tool_calls) that represents the tool call in a format compatible with OpenAI's tool calling API.

It generates a unique ID using uuid.uuid4().

It sets the type to "function".

It extracts the tool's name from tool_call.__class__.__name__ (assuming tool_call is an object with a class name).

It converts the tool's arguments to a JSON string using tool_call.json().

The tool call information is appended to the openai_tool_calls list.

AI Message with Tool Calls:

Creates an AIMessage object with an empty content and adds the openai_tool_calls to its additional_kwargs. This represents the AI's response, which includes the tool calls.

Tool Output Handling:

Retrieves the tool_outputs from the example dictionary. If tool_outputs is not provided, it defaults to a list of "You have correctly called this tool." messages, with a length that matches the number of tool calls.

Iterates through the tool_outputs and openai_tool_calls lists in parallel.
For each output and tool call, it creates a ToolMessage object, adding it to the messages list. This represents the result of the tool call.

Return: Returns the messages list.

3. example_msgs Creation:

Purpose: Creates a flattened list of messages from a list of examples.

Steps:

Uses a list comprehension to iterate through a list named examples (which is assumed to be defined elsewhere).

For each ex in examples, it calls tool_example_to_messages(ex) to convert the example to a list of messages.

It then flattens the resulting list of lists into a single list named example_msgs.

Summary:

This code is a utility function used to convert data from a specific format into a list of messages. This is common when using LangChain to train or test a language model that uses tools. The code effectively transforms a structured representation of a conversation into a sequence of messages that LangChain can understand and process.

In [10]:
import uuid
from typing import Dict

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
)


def tool_example_to_messages(example: Dict) -> List[BaseMessage]:
    messages: List[BaseMessage] = [HumanMessage(content=example["input"])]
    openai_tool_calls = []
    for tool_call in example["tool_calls"]:
        openai_tool_calls.append(
            {
                "id": str(uuid.uuid4()),
                "type": "function",
                "function": {
                    "name": tool_call.__class__.__name__,
                    "arguments": tool_call.json(),
                },
            }
        )
    messages.append(
        AIMessage(content="", additional_kwargs={"tool_calls": openai_tool_calls})
    )
    tool_outputs = example.get("tool_outputs") or [
        "You have correctly called this tool."
    ] * len(openai_tool_calls)
    for output, tool_call in zip(tool_outputs, openai_tool_calls):
        messages.append(ToolMessage(content=output, tool_call_id=tool_call["id"]))
    return messages


example_msgs = [msg for ex in examples for msg in tool_example_to_messages(ex)]

<ipython-input-10-58cddf997614>:23: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  "arguments": tool_call.json(),


Summary:

This code defines a chain that:

Receives a question as input (within a dictionary).

Prepares a prompt that includes the question and a set of example messages.
Sends the prepared prompt to a structured language model.

Receives and returns the structured response from the language model.

This is a common pattern for building question-answering or query-analysis systems that learn from examples. The examples provide context and guide the language model in generating appropriate responses.

In [11]:
from langchain_core.prompts import MessagesPlaceholder

query_analyzer_with_examples = (
    {"question": RunnablePassthrough()}
    | prompt.partial(examples=example_msgs)
    | structured_llm
)

In [13]:
query_analyzer_with_examples.invoke(
    "what's the difference between web voyager and reflection agents? do both use langgraph?"
)

Search(query="What's the difference between web voyager and reflection agents? Do both use langgraph?", sub_queries=['What is web voyager?', 'What are reflection agents?', 'What is the difference between web voyager and reflection agents?', 'Do web voyager and reflection agents use langgraph?'], publish_year=None)

In [14]:
query_analyzer_with_examples.invoke(
    "what's the difference between Agentic RAG and Corrective RAG ?"
)

Search(query='What is the difference between Agentic RAG and Corrective RAG?', sub_queries=['What is Agentic RAG?', 'What is Corrective RAG?', 'How do Agentic RAG and Corrective RAG differ?'], publish_year=None)

In [15]:
query_analyzer_with_examples.invoke(
    "what's the difference between RAG's and Chatbots ?"
)

Search(query='What is the difference between RAGs and chatbots?', sub_queries=['What are RAGs?', 'What are chatbots?', 'How do RAGs work compared to chatbots?'], publish_year=None)